# Лабораторная работа №1

## Содержание:

- найти велосипед с максимальным временем пробега
- найти наибольшей геодезическое расстояние между станциями
- найти путь велосипеда с максимальным временем пробега через станции
- найти количество велосипедов в системе
- найти пользователей потративших на поездки более 3 часов

### 1. Импортируем библиотеки и изучаем датасеты

In [8]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as t
from geopy.distance import geodesic
import pandas as pd
from itertools import combinations

In [9]:
def create_spark_session():
    """Создание и настройка SparkSession."""
    return SparkSession.builder \
        .appName("CSV Data Processing") \
        .getOrCreate()


def load_csv_data(spark_session, file_path):
    """Чтение CSV файла в DataFrame."""
    return spark_session.read.format('csv') \
        .option('header', 'true') \
        .load(file_path)


spark_session = create_spark_session()

# Чтение данных из CSV файлов
station_data = load_csv_data(spark_session, 'station.csv')
trip_data = load_csv_data(spark_session, 'trip.csv')

station_data.show(5)
trip_data.show(5)


+---+--------------------+------------------+-------------------+----------+--------+-----------------+
| id|                name|               lat|               long|dock_count|    city|installation_date|
+---+--------------------+------------------+-------------------+----------+--------+-----------------+
|  2|San Jose Diridon ...|         37.329732|-121.90178200000001|        27|San Jose|         8/6/2013|
|  3|San Jose Civic Ce...|         37.330698|        -121.888979|        15|San Jose|         8/5/2013|
|  4|Santa Clara at Al...|         37.333988|        -121.894902|        11|San Jose|         8/6/2013|
|  5|    Adobe on Almaden|         37.331415|          -121.8932|        19|San Jose|         8/5/2013|
|  6|    San Pedro Square|37.336721000000004|        -121.894074|        15|San Jose|         8/7/2013|
+---+--------------------+------------------+-------------------+----------+--------+-----------------+
only showing top 5 rows

+----+--------+--------------+---------

### 2. Задания

#### 2.1  Найти велосипед с максимальным временем пробега

In [10]:
from pyspark.sql import functions as F

# Grouping by bike_id and summing the duration for each bike
bike_total_duration_df = trip_data.groupBy("bike_id").agg(
    F.sum("duration").alias("total_duration")
)

# Finding the bike with the maximum total duration
max_duration_bike = bike_total_duration_df.orderBy(F.desc("total_duration")).limit(1).collect()[0]

print(f"The bike with ID {max_duration_bike['bike_id']} has the maximum total duration: {max_duration_bike['total_duration']} seconds.")

The bike with ID 535 has the maximum total duration: 18611693.0 seconds.


#### 2.2 Найти наибольшей геодезическое расстояние между станциями

In [ ]:
from pyspark.sql import SparkSession
from geopy.distance import geodesic
from itertools import combinations
import pandas as pd

spark = SparkSession.builder \
    .appName("Max Geodesic Distance") \
    .getOrCreate()

stations_df = spark.read.format('csv').option('header', 'true').load('station.csv')

# Преобразуем широту и долготу в числовой формат
stations_df = stations_df.withColumn("lat", stations_df["lat"].cast("float")) \
                          .withColumn("long", stations_df["long"].cast("float"))

# Собираем все станции в список
stations = stations_df.select("id", "lat", "long").collect()

# Вычисление расстояний между всеми парами станций с использованием geopy
max_distance = 0
station_pair = None
distance_list = []

for station1, station2 in combinations(stations, 2):
    # Получаем координаты двух станций
    coord1 = (station1['lat'], station1['long'])
    coord2 = (station2['lat'], station2['long'])
    
    # Вычисляем геодезическое расстояние
    distance = geodesic(coord1, coord2).kilometers
    
    distance_list.append({
        'station_1_id': station1['id'],
        'station_2_id': station2['id'],
        'distance_km': distance
    })
    
    # Обновляем максимальное расстояние и соответствующую пару станций
    if distance > max_distance:
        max_distance = distance
        station_pair = (station1['id'], station2['id'])


# Выводим результат с максимальным расстоянием
print(f"The maximum geodesic distance is between stations {station_pair[0]} and {station_pair[1]} with a distance of {max_distance} kilometers.")

The maximum geodesic distance is between stations 16 and 60 with a distance of 69.9212704912406 kilometers.


### 2.3 Найти путь велосипеда с максимальным временем пробега через станции

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import unix_timestamp, col
from pyspark.sql import functions as F
from itertools import combinations

spark = SparkSession.builder \
    .appName("Max Geodesic Distance") \
    .getOrCreate()

trips_df_spark = trip_data

# Преобразуем даты в метки времени и считаем длительность поездки в минутах
trips_with_duration = trips_df_spark.withColumn(
    "start_timestamp", unix_timestamp(col("start_date"), "M/d/yyyy H:mm")
).withColumn(
    "end_timestamp", unix_timestamp(col("end_date"), "M/d/yyyy H:mm")
).withColumn(
    "duration_minutes", (col("end_timestamp") - col("start_timestamp")) / 60
)

# Группируем по bike_id и находим велосипед с максимальным временем пробега
bike_with_max_duration = trips_with_duration.groupBy("bike_id").agg(
    F.sum("duration_minutes").alias("total_duration_minutes")
).orderBy(F.desc("total_duration_minutes")).limit(1)

# Получаем bike_id с максимальным временем пробега
max_bike_id = bike_with_max_duration.collect()[0]["bike_id"]

# Извлекаем все поездки для этого велосипеда
bike_trips = trips_with_duration.filter(col("bike_id") == max_bike_id)

# Сортируем поездки по времени начала
bike_trips_sorted = bike_trips.orderBy("start_timestamp")

# Извлекаем последовательность станций
path_stations = bike_trips_sorted.select("start_station_name", "end_station_name").collect()

# Формируем путь через станции
path = []
for trip in path_stations:
    path.append(trip["start_station_name"])
    path.append(trip["end_station_name"])

# Убираем дублирующиеся станции (если есть)
path = list(dict.fromkeys(path))

print(f"Path for the bike with ID {max_bike_id}:")
print(" -> \n".join(path))


Path for the bike with ID 535:
Post at Kearney -> 
San Francisco Caltrain (Townsend at 4th) -> 
San Francisco Caltrain 2 (330 Townsend) -> 
Market at Sansome -> 
2nd at South Park -> 
2nd at Townsend -> 
Davis at Jackson -> 
San Francisco City Hall -> 
Civic Center BART (7th at Market) -> 
Embarcadero at Sansome -> 
Washington at Kearney -> 
2nd at Folsom -> 
Temporary Transbay Terminal (Howard at Beale) -> 
Clay at Battery -> 
Harry Bridges Plaza (Ferry Building) -> 
Steuart at Market -> 
Townsend at 7th -> 
5th at Howard -> 
Mechanics Plaza (Market at Battery) -> 
Powell at Post (Union Square) -> 
Market at 4th -> 
Market at 10th -> 
Beale at Market -> 
Golden Gate at Polk -> 
Powell Street BART -> 
Spear at Folsom -> 
Embarcadero at Vallejo -> 
Commercial at Montgomery -> 
Yerba Buena Center of the Arts (3rd @ Howard) -> 
Grant Avenue at Columbus Avenue -> 
Howard at 2nd -> 
Embarcadero at Bryant -> 
Embarcadero at Folsom -> 
South Van Ness at Market -> 
Broadway St at Battery St ->

### 2.4 Найти количество велосипедов в системе

In [27]:
unique_bikes_count_spark = trips_df_spark.select("bike_id").distinct().count()

# Выводим результат
print(f"The number of unique bikes in the system is: {unique_bikes_count_spark}")

The number of unique bikes in the system is: 700


### 2.5 Найти пользователей потративших на поездки более 3 часов

In [ ]:
# Преобразуем даты в метки времени и считаем длительность поездки в минутах
trips_with_duration = trips_df_spark.withColumn(
    "start_timestamp", unix_timestamp("start_date", "M/d/yyyy H:mm")
).withColumn(
    "end_timestamp", unix_timestamp("end_date", "M/d/yyyy H:mm")
).withColumn(
    "duration_minutes", (col("end_timestamp") - col("start_timestamp")) / 60
)

# Суммируем длительность поездок для каждого bike_id
user_trip_duration = trips_with_duration.groupBy("bike_id").agg(
    F.sum("duration_minutes").alias("total_duration_minutes")
)

# Фильтруем пользователей, у которых суммарное время на поездках больше 180 минут (3 часа)
users_above_3_hours = user_trip_duration.filter(col("total_duration_minutes") > 180)

users_above_3_hours.show()


+-------+----------------------+
|bike_id|total_duration_minutes|
+-------+----------------------+
|    675|                6189.0|
|    467|               15185.0|
|    296|                8117.0|
|    691|                4921.0|
|    125|                2633.0|
|    451|               28274.0|
|    666|                2792.0|
|    447|               25823.0|
|    124|                6082.0|
|    591|               34620.0|
|     51|                6460.0|
|    574|               32415.0|
|    613|               40114.0|
|    307|                6083.0|
|    544|               27289.0|
|    334|               31621.0|
|    577|               27304.0|
|    581|               30337.0|
|    205|                2586.0|
|    647|                4100.0|
+-------+----------------------+
only showing top 20 rows

